In [ ]:
import os
import pandas as pd
import numpy as np
import kagglehub
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from IPython.display import FileLink
from google.colab import files

Chargement et Compréhension du dataset

In [ ]:
# Download latest version
path = kagglehub.dataset_download("carrie1/ecommerce-data")
file_path = os.path.join(path, "data.csv")
data = pd.read_csv(file_path, header=0, on_bad_lines= 'skip', encoding= 'latin1' )

100%|██████████| 7.20M/7.20M [00:00<00:00, 34.8MB/s]

Extracting files...


In [ ]:
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [ ]:
data.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
#________________________________________________________________nettoyage du dataframe_____________________________________________________________
data_clean = data.copy()
#Vérification de l'unicité du stockcode pour un produit
data_clean[data_clean['StockCode'] == '85123A']['Description'].unique()


array(['WHITE HANGING HEART T-LIGHT HOLDER', '?',
       'wrongly marked carton 22804',
       'CREAM HANGING HEART T-LIGHT HOLDER'], dtype=object)

Un stockcode peut appartenir à plusieurs produits

In [ ]:
#Vérification de l'unicité du prix pour chaque produit ayant le même stockcode stockcode
data_clean[data_clean['StockCode'] == '85123A']['UnitPrice'].unique()

array([2.55, 2.95, 5.91, 0.  , 3.24, 5.79, 2.4 , 3.2 ])

Les produits ayant le même stockcode ont généralement pas le même prix

In [ ]:
#Remplissage des descriptions manquantes
# On remplace les descriptions vides par le mode des descriptions qui ont le même UnitPrice et le même StockCode
data_clean['Description'] = data_clean.groupby(['StockCode', 'UnitPrice'])['Description'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Unknown")) #récupération de quelques descriptions
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  541909 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
data_clean = data_clean[data_clean['Description'] != 'Unknown'] # Suppression des colonnes qui auront Unknown en description

In [ ]:
#Suppression des valeurs manquantes
data_clean.dropna(inplace=True)
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [ ]:
#Verification d'existence de doublons
data_clean.duplicated().sum()

5268

In [ ]:
#Suppression des doublons
data_clean.drop_duplicates(inplace=True)

In [ ]:
#Vérification de l'existence de quantités négatives
data_clean[data_clean['Quantity'] < 0].info()

<class 'pandas.core.frame.DataFrame'>
Index: 8872 entries, 141 to 541717
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    8872 non-null   object 
 1   StockCode    8872 non-null   object 
 2   Description  8872 non-null   object 
 3   Quantity     8872 non-null   int64  
 4   InvoiceDate  8872 non-null   object 
 5   UnitPrice    8872 non-null   float64
 6   CustomerID   8872 non-null   float64
 7   Country      8872 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 623.8+ KB


In [ ]:
#Suppression des données avec des quantités négatives
data_clean = data_clean[data_clean['Quantity'] > 0]

Data clean contient maintenant notre dataset propre et prêt à être transformé puis analyser

In [ ]:
#__________________________________________________________________TRANSFORMATION DES DONNEES________________________________________________________________
data_clean.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
224504,556531,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,4,6/13/2011 11:49,4.95,15491.0,United Kingdom
298816,563061,22024,RAINY LADIES BIRTHDAY CARD,12,8/11/2011 15:25,0.42,13588.0,United Kingdom
342681,566912,21166,COOK WITH WINE METAL SIGN,12,9/15/2011 14:05,2.08,14367.0,United Kingdom
498020,578508,22142,CHRISTMAS CRAFT WHITE FAIRY,3,11/24/2011 13:06,1.45,16892.0,United Kingdom
118396,546429,22332,SKULLS PARTY BAG + STICKER SET,6,3/13/2011 15:25,1.65,17611.0,United Kingdom
225353,556671,21784,SHOE SHINE BOX,1,6/13/2011 16:33,9.95,17841.0,United Kingdom
140614,548451,22813,PACK 3 BOXES BIRD PANNETONE,2,3/31/2011 11:25,1.95,17365.0,United Kingdom
82098,543188,22962,JAM JAR WITH PINK LID,12,2/4/2011 11:45,0.85,12567.0,France
204142,554636,23076,ICE CREAM SUNDAE LIP GLOSS,24,5/25/2011 11:52,1.25,16763.0,United Kingdom
454302,575516,20750,RED RETROSPOT MINI CASES,6,11/10/2011 10:45,7.95,17340.0,United Kingdom


In [ ]:
# conversion de la data en date
data_clean['InvoiceDate'] = pd.to_datetime(data_clean['InvoiceDate'])

In [ ]:
#Ajout de colonne contenant les jours, les mois, les années, les heures, et jours de la semaines auquels des commandes ont été passées
#Colonne Day
data_clean['Day'] = data_clean['InvoiceDate'].dt.strftime('%Y-%m-%d')
#colonne month
data_clean['Month'] = data_clean['InvoiceDate'].dt.strftime('%B')
#Colonne year
data_clean['Year'] = data_clean['InvoiceDate'].dt.strftime('%Y')
#Colonne heure
data_clean['Hour'] = data_clean['InvoiceDate'].dt.strftime('%H:%M')
#Colonne days
data_clean['Week_day'] = data_clean['InvoiceDate'].dt.strftime('%A')

In [ ]:
#Ajout d'une colonne Montant total
data_clean['MontantTotal'] = data_clean['Quantity'] * data_clean['UnitPrice']

In [ ]:
#Changement du type pour les id des clients
data_clean['CustomerID'] = data_clean['CustomerID'].astype(str)

In [ ]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392732 entries, 0 to 541908
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     392732 non-null  object        
 1   StockCode     392732 non-null  object        
 2   Description   392732 non-null  object        
 3   Quantity      392732 non-null  int64         
 4   InvoiceDate   392732 non-null  datetime64[ns]
 5   UnitPrice     392732 non-null  float64       
 6   CustomerID    392732 non-null  object        
 7   Country       392732 non-null  object        
 8   Day           392732 non-null  object        
 9   Month         392732 non-null  object        
 10  Year          392732 non-null  object        
 11  Hour          392732 non-null  object        
 12  Week_day      392732 non-null  object        
 13  MontantTotal  392732 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(10)
memory usage: 44.9

ANALYSE DES DONNEES

Verification de l'existence d'outlet

In [ ]:
data_quantitatives = data_clean.select_dtypes(include='number')
data_quantitatives

,Quantity,UnitPrice,MontantTotal
0,6,2.55,15.30
1,6,3.39,20.34
2,8,2.75,22.00
3,6,3.39,20.34
4,6,3.39,20.34
...,...,...,...
541904,12,0.85,10.20
541905,6,2.10,12.60
541906,4,4.15,16.60
541907,4,4.15,16.60


In [ ]:
data_quantitatives.describe()

NameError: name 'data_quantitatives' is not defined

In [ ]:
#Les quartiles 10 pour les prix unitaires m'aideront à comprendre la tranche de prix dans lesquels les produits sont les plus achetés
quantile_prix = data_quantitatives['UnitPrice'].quantile(q = [_/100 for _ in range (0,101,1)])
quantile_prix.index = [f"{(q * 100)}%" for q in quantile_prix.index]
quantile_prix

NameError: name 'data_quantitatives' is not defined

99% des articles vendus ont un prix compris entre 0,55 et 14,95
Seulement 1% des articles vendus ont un prix supérieur à 14,95

Dans un premier temps, je vais garder un dataset avec les outlets pour voir les produits de luxe qui peuvent être interessant à vendre et assi les pays dans lesquels il peut le plus interessant de les vendre.

In [ ]:
#dataset avec les valeurs abberantes
data_produit_luxe = data_clean[data_clean['UnitPrice'] > 14.95]
data_produit_luxe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3723 entries, 45 to 541768
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   InvoiceNo     3723 non-null   object        
 1   StockCode     3723 non-null   object        
 2   Description   3723 non-null   object        
 3   Quantity      3723 non-null   int64         
 4   InvoiceDate   3723 non-null   datetime64[ns]
 5   UnitPrice     3723 non-null   float64       
 6   CustomerID    3723 non-null   object        
 7   Country       3723 non-null   object        
 8   Day           3723 non-null   object        
 9   Month         3723 non-null   object        
 10  Year          3723 non-null   object        
 11  Hour          3723 non-null   object        
 12  Week_day      3723 non-null   object        
 13  MontantTotal  3723 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(10)
memory usage: 436.3+ KB


In [ ]:
#Les 10 produits de luxe les plus vendus
vente_luxe_max = data_produit_luxe.groupby('Description').agg({'Quantity':'sum'}).reset_index().sort_values(by='Quantity', ascending=False)
top_10_produit_luxe = vente_luxe_max.head(10)
top_10_produit_luxe

NameError: name 'data_produit_luxe' is not defined

In [ ]:
# Pays dans lesquels les produits de luxes sont les plus vendus
data_best_luxe_country = data_produit_luxe[data_produit_luxe['Description'].isin(top_10_produit_luxe['Description'])] #dataframe avec les pays dans lesquels il y a eu achat des 10 produits de luxe
groupe_best_luxe_country = data_best_luxe_country.groupby('Country').agg({'Quantity':'sum'}).reset_index().sort_values(by='Quantity', ascending=False)
top_10_groupe_best_luxe_country = groupe_best_luxe_country.head(10)
top_10_groupe_best_luxe_country

,Country,Quantity
24,United Kingdom,1539
12,Germany,1136
11,France,877
2,Belgium,306
20,Spain,224
8,EIRE,188
22,Switzerland,119
16,Netherlands,108
19,Portugal,97
10,Finland,96


In [ ]:
# mois enregistrant le plus de ventes de produits de luxe
data_month_sell_luxe = data_produit_luxe.groupby('Month').agg({'Quantity':'sum'}).reset_index()
#ajout d'une colonne qui va contenir les pourcentages
data_month_sell_luxe['Pourcentage'] = (data_month_sell_luxe['Quantity'] / data_month_sell_luxe['Quantity'].sum()) * 100
#ajout d'une colonne qui va conenir la correspondance des mois en chiffre
month_number = np.array([4,8,12,2,1,7,6,3,5,11,10,9]) # ordre des mois lorsqu'on fera des group by month
data_month_sell_luxe['Month_number'] = month_number
data_month_sell_luxe = data_month_sell_luxe.sort_values(by='Month_number')
data_month_sell_luxe

,Month,Quantity,Pourcentage,Month_number
4,January,480,6.309148,1
3,February,335,4.403260,2
7,March,544,7.150368,3
0,April,338,4.442692,4
8,May,634,8.333333,5
6,June,637,8.372766,6
5,July,522,6.861199,7
1,August,467,6.138275,8
11,September,603,7.925868,9
10,October,1027,13.498948,10


A cette étape, j'ai les 10 pays dans lesquels on peut le plus vendre les produits de luxe, les 10 produis de luxe qui se vendent le plus et la période de l'année à laquelle ils se vendent le plus

In [ ]:
#___________________________________________________________________Dataframe sans les outlers_____________________________________________________________________
data_reduit = data_clean[data_clean['UnitPrice']<= 14.95]
data_reduit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 389009 entries, 0 to 541908
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     389009 non-null  object        
 1   StockCode     389009 non-null  object        
 2   Description   389009 non-null  object        
 3   Quantity      389009 non-null  int64         
 4   InvoiceDate   389009 non-null  datetime64[ns]
 5   UnitPrice     389009 non-null  float64       
 6   CustomerID    389009 non-null  object        
 7   Country       389009 non-null  object        
 8   Day           389009 non-null  object        
 9   Month         389009 non-null  object        
 10  Year          389009 non-null  object        
 11  Hour          389009 non-null  object        
 12  Week_day      389009 non-null  object        
 13  MontantTotal  389009 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(10)
memory usage: 44.5

In [ ]:
# Nombre de produits vendus par mois
mois_vente = data_reduit.groupby('Month').agg({'Quantity':'sum'}).reset_index() #dataframe contenant les mois regroupés et le nombre de produits vendu par mois
#numero_mois = np.array([11,12,10,9,8,5,7,6,1,3,4,2])
mois_vente['Month_number'] = month_number
mois_vente = mois_vente.sort_values(by='Month_number')
mois_vente

,Month,Quantity,Month_number
4,January,348042,1
3,February,264708,2
7,March,347079,3
0,April,291031,4
8,May,372314,5
6,June,362377,6
5,July,366850,7
1,August,397723,8
11,September,543051,9
10,October,590524,10


In [ ]:
#chiffre d'affaire par mois
chiffre_affaire_mois = data_reduit.groupby('Month').agg({'MontantTotal':'sum'}).reset_index()
chiffre_affaire_mois['Month_number'] = month_number
chiffre_affaire_mois = chiffre_affaire_mois.sort_values(by='Month_number')
chiffre_affaire_mois

,Month,MontantTotal,Month_number
4,January,554556.460,1
3,February,436379.080,2
7,March,573527.920,3
0,April,448738.261,4
8,May,649545.090,5
6,June,604849.070,6
5,July,584305.771,7
1,August,629893.600,8
11,September,931280.202,9
10,October,989587.090,10


Les chiffres d'affaires suivent bien la quantité des produits vendus par mois. DOnc il nous faut vendre beaucoup de produits

In [ ]:
#Liste des produits qui se vendent les plus
nombre_de_vente_par_produit = data_reduit.groupby('Description').agg({'Quantity':'sum'}).reset_index()
top_20_produit = nombre_de_vente_par_produit.sort_values(by='Quantity', ascending=False).head(20)
top_20_produit

,Description,Quantity
2298,"PAPER CRAFT , LITTLE BIRDIE",80995
1972,MEDIUM CERAMIC TOP STORAGE JAR,77916
3747,WORLD WAR 2 GLIDERS ASSTD DESIGNS,54319
1744,JUMBO BAG RED RETROSPOT,46078
3660,WHITE HANGING HEART T-LIGHT HOLDER,36706
213,ASSORTED COLOUR BIRD ORNAMENT,35263
2248,PACK OF 72 RETROSPOT CAKE CASES,33670
2577,POPCORN HOLDER,30919
2633,RABBIT NIGHT LIGHT,27153
2027,MINI PAINT SET VINTAGE,26076


In [ ]:
# Dependances du prix unitaire et de la quantité
# Définition des variables
x = data_reduit['Quantity']
y = data_reduit['UnitPrice']
relation = stats.pearsonr(data_reduit['Quantity'], data_reduit['UnitPrice'])
relation

PearsonRResult(statistic=-0.03011113619775141, pvalue=1.0077147075034089e-78)

In [ ]:
# Calcul de la régression linéaire
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Génération des points pour la droite
x_fit = np.linspace(x.min(), x.max(), 100)
y_fit = slope * x_fit + intercept

In [ ]:
#Graphique de la droite de regression lineaire 8
# Création du scatter plot
fig = px.scatter(data_reduit, x="Quantity", y="UnitPrice",
                 title="Corrélation entre Quantity et UnitPrice",
                 labels={"Quantity": "Quantité", "UnitPrice": "Prix Unitaire"},
                 opacity=0.6)

# Ajout de la droite de régression
fig.add_trace(go.Scatter(x=x_fit, y=y_fit, mode="lines",
                         name=f"Régression linéaire (r={r_value:.2f})",
                         line=dict(color="red", width=2)))

# Affichage du graphique
fig.show()

In [ ]:
#prix des produits qui se vendent le plus
data_best_product_price = data_reduit[data_reduit['Description'].isin(top_20_produit['Description'])] #dataframe avec que les transactions pour les tops produits
data_best_product_price = data_best_product_price.groupby('Description').agg({'UnitPrice':'mean'}).reset_index()# Les tops produits et la moyenne de leur prix
data_best_product_price = data_best_product_price.sort_values(by='UnitPrice', ascending=True)
data_best_product_price

,Description,UnitPrice
19,WORLD WAR 2 GLIDERS ASSTD DESIGNS,0.292606
3,BROCADE RING PURSE,0.320240
9,PACK OF 12 LONDON TISSUES,0.325816
0,60 TEATIME FAIRY CAKE CASES,0.548170
11,PACK OF 72 RETROSPOT CAKE CASES,0.548181
10,PACK OF 60 PINK PAISLEY CAKE CASES,0.551420
8,MINI PAINT SET VINTAGE,0.656523
2,ASSORTED COLOURS SILK FAN,0.805015
16,SMALL POPCORN HOLDER,0.832752
13,POPCORN HOLDER,0.843231


In [ ]:
#Je veux vérifier si la vente d'un produit dépend du pays
tableau = pd.crosstab(data_reduit['Description'], data_reduit['Country']) # tableau de contingence
chi2, p, dof, expected = stats.chi2_contingency(tableau) # Utilisation d Chi2 contingency pour voir la P value
p

0.0

La p value est de 0 donc l'hypothèse selon laquelle les 2 variables sont indépendantes est rejetée.

In [ ]:
#Les pays dans lesquels les top produits sont les plus vendus
data_best_country = data_reduit[data_reduit['Description'].isin(top_20_produit['Description'])] #dataframe ne contenant que les produits du top 20
data_best_country = data_best_country.groupby('Country').agg({'Quantity':'sum'}).reset_index().sort_values(by='Quantity', ascending=False)
top_10_data_best_country = data_best_country.head(10)

Les mois qui enregistrent les plus de ventes sont les mois les plus rentables, donc on doit s'assurer de vendre beaucoup de produits

In [ ]:
# La liste des produits qui se vendent le moins
vente_min = nombre_de_vente_par_produit.sort_values(by='Quantity', ascending=True)
vente_min.head(50)

,Description,Quantity
2204,PACK 4 FLOWER/BUTTERFLY PATCHES,1
2105,MUMMY MOUSE RED GINGHAM RIBBON,1
1712,JARDIN ETCHED GLASS BUTTER DISH,1
441,BLUE LEAVES AND BEADS PHONE CHARM,1
675,CHERRY BLOSSOM PURSE,1
447,BLUE NEW BAROQUE FLOCK CANDLESTICK,1
3088,SET/3 TALL GLASS CANDLE HOLDER PINK,1
1704,JADE CRYSTAL+GLASS BRACELET,1
450,BLUE PADDED SOFT MOBILE,1
3081,SET/3 FLORAL GARDEN TOOLS IN BAG,1


In [ ]:
#Les clients qui depensent le plus
clients_rentables = data_reduit.groupby('CustomerID').agg({'MontantTotal':'sum'}).reset_index().sort_values(by='MontantTotal', ascending=False) # dataset des dépenses par cliients
clients_rentables = clients_rentables[clients_rentables['MontantTotal'] > 500] # On garde ceux qui ont dpensés 500
clients_rentables['CustomerID'] = clients_rentables['CustomerID'].astype(str) # conversion de CustomerID
top_20_clients_rentables = clients_rentables.head(20)
top_20_clients_rentables


,CustomerID,MontantTotal
1688,14646.0,278349.22
4192,18102.0,250414.48
3721,17450.0,194390.79
3002,16446.0,168472.50
1878,14911.0,134052.38
55,12415.0,124564.53
1332,14156.0,114818.28
3764,17511.0,91011.53
0,12346.0,77183.60
2697,16029.0,72708.09


In [ ]:
#clients qui achetent plusieurs fois
clients_achat_multiple = data_reduit.groupby('CustomerID').agg({'InvoiceNo':'nunique'}).reset_index().sort_values(by='InvoiceNo', ascending=False) # regroupe les clients en comptant le nombres de factures qui leur appartient
clients_achat_multiple = clients_achat_multiple[clients_achat_multiple['InvoiceNo'] > 1] # On garde les clients qui ont achetés plus d'une fois
clients_achat_multiple['CustomerID'] = clients_achat_multiple['CustomerID'].astype(str) # conversion en string
top_20_clients_achat_multiple = clients_achat_multiple.head(20) # les 20 premiers de la liste
top_20_clients_achat_multiple

,CustomerID,InvoiceNo
326,12748.0,206
1878,14911.0,198
4003,17841.0,124
561,13089.0,95
2175,15311.0,91
1660,14606.0,90
480,12971.0,86
1688,14646.0,73
2697,16029.0,62
795,13408.0,62


In [ ]:
# Les produits qui se vendent le plus en Mars
data_mars = data_reduit[data_reduit['Month'] == 'March']
vente_mars = data_mars.groupby('Description').agg({'Quantity':'sum'}).reset_index().sort_values(by='Quantity', ascending=False)
vente_mars = vente_mars.head(20)

In [ ]:
#Les clients qui achetent plusieurs fois et qui dépensent le plus
clients_fideles_et_rentables = pd.merge(clients_rentables, clients_achat_multiple, on="CustomerID", how="inner") # jointure des datasets clients_rentables et clients_achats_multiple
clients_fideles_et_rentables['CustomerID'] = clients_fideles_et_rentables['CustomerID'].astype(str) #conversion des CustomerID en string
clients_fideles_et_rentables_20 = clients_fideles_et_rentables.head(20) # les 20 premiers clients
clients_fideles_et_rentables_20

,CustomerID,MontantTotal,InvoiceNo
0,14646.0,278349.22,73
1,18102.0,250414.48,59
2,17450.0,194390.79,46
3,16446.0,168472.50,2
4,14911.0,134052.38,198
5,12415.0,124564.53,20
6,14156.0,114818.28,53
7,17511.0,91011.53,31
8,16029.0,72708.09,62
9,16684.0,66653.56,28


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Création des sous-graphiques avec une seule colonne et plusieurs lignes
fig = make_subplots(
    rows=14, cols=1,  # 13 graphiques en 1 colonne
    subplot_titles=[
        "Distribution des prix unitaires par quantiles",
        "Top 10 produits de luxe les plus vendus",
        "Top 10 pays dans lesquels il y a le plus de ventes pour les produits de luxe",
        "Mois enregistrant le plus de ventes de produits de luxe",
        "Mois avec le plus de ventes",
        "Chiffre d'affaires par mois",
        "Top 20 des produits commerciaux",
        "Prix Unitaire des tops produits",
        "Meilleurs pays pour la vente de produits commerciaux",
        "Produits les moins vendus",
        "Clients les plus rentables",
        "Clients qui achètent plusieurs fois",
        "Clients qui depensent le plus et en plusieurs fois",
        "Produits qui se vendent le plus en Mars",
    ]
)

# Ajout du premier graphique : Distribution des prix unitaires par quantiles
fig.add_trace(
    go.Scatter(x=quantile_prix.index, y=quantile_prix.values, mode='lines+markers'),
    row=1, col=1
)

# Graphique 2 : Top 10 produits de luxe les plus vendus
fig.add_trace(
    go.Bar(x=top_10_produit_luxe['Description'], y=top_10_produit_luxe['Quantity']),
    row=2, col=1
)

# Graphique 3 : Top 10 pays ayant le plus de ventes pour les produits de luxe
fig.add_trace(
    go.Bar(x=top_10_groupe_best_luxe_country['Country'], y=top_10_groupe_best_luxe_country['Quantity']),
    row=3, col=1
)

# Graphique 4 : Mois enregistrant le plus de ventes de produits de luxe
fig.add_trace(
    go.Bar(x=data_month_sell_luxe['Month'], y=data_month_sell_luxe['Pourcentage']),
    row=4, col=1
)

# Graphique 5 : Mois avec le plus de ventes
fig.add_trace(
    go.Bar(x=mois_vente['Month'], y=mois_vente['Quantity']),
    row=5, col=1
)

# Graphique 6 : Chiffre d'affaires par mois
fig.add_trace(
    go.Scatter(x=chiffre_affaire_mois['Month'], y=chiffre_affaire_mois['MontantTotal'], mode='lines+markers'),
    row=6, col=1
)

# Graphique 7 : Top 20 des produits commerciaux
fig.add_trace(
    go.Bar(x=top_20_produit['Description'], y=top_20_produit['Quantity']),
    row=7, col=1
)

# Graphique 8 : Prix Unitaire des tops produits
fig.add_trace(
    go.Bar(x=data_best_product_price['Description'], y=data_best_product_price['UnitPrice']),
    row=8, col=1
)

# Graphique 9 : Meilleurs pays pour la vente de produits commerciaux
fig.add_trace(
    go.Bar(x=top_10_data_best_country['Country'], y=top_10_data_best_country['Quantity']),
    row=9, col=1
)

# Graphique 10 : Produits les moins vendus
fig.add_trace(
    go.Bar(x=vente_min['Description'], y=vente_min['Quantity']),
    row=10, col=1
)

# Graphique 11 : Clients les plus rentables
fig.add_trace(
    go.Bar(x=top_20_clients_rentables['CustomerID'], y=top_20_clients_rentables['MontantTotal']),
    row=11, col=1
)

# Graphique 12 : Clients qui achètent plusieurs fois
fig.add_trace(
    go.Bar(x=top_20_clients_achat_multiple['CustomerID'], y=top_20_clients_achat_multiple['InvoiceNo']),
    row=12, col=1
)

# Graphique 13: Clients qui depensent le plus et en plusieurs fois
fig.add_trace(
    go.Bar(x=clients_fideles_et_rentables_20['CustomerID'], y=clients_fideles_et_rentables_20['MontantTotal']),
    row=13, col=1
)

# Graphique 14 : Produits qui se vendent le plus en Mars
fig.add_trace(
    go.Bar(x=vente_mars['Description'], y=vente_mars['Quantity']),
    row=14, col=1
)

# Mettre à jour la mise en page générale
fig.update_layout(
    height=10000,  #l'espace vertical
    width=1200,
    title_text="Rapport d'Analyse des Données avec Graphiques Plotly",
    showlegend=False
)


# Mise à jour des axes
fig.update_xaxes(title_text="Quantiles", row=1, col=1)
fig.update_yaxes(title_text="Prix Unitaire (log)", type="log", row=1, col=1)

fig.update_xaxes(title_text="Produit", row=2, col=1)
fig.update_yaxes(title_text="Quantité vendue", row=2, col=1)

fig.update_xaxes(title_text="Pays", row=3, col=1)
fig.update_yaxes(title_text="Quantité vendue", row=3, col=1)

fig.update_xaxes(title_text="Mois", row=4, col=1)
fig.update_yaxes(title_text="Pourcentage de ventes", row=4, col=1)

fig.update_xaxes(title_text="Mois", row=5, col=1)
fig.update_yaxes(title_text="Quantité vendue", row=5, col=1)

fig.update_xaxes(title_text="Mois", row=6, col=1)
fig.update_yaxes(title_text="Chiffre d'affaires", row=6, col=1)

fig.update_xaxes(title_text="Produit", row=7, col=1)
fig.update_yaxes(title_text="Quantité vendue", row=7, col=1)

fig.update_xaxes(title_text="Produit", row=8, col=1)
fig.update_yaxes(title_text="Prix Unitaire", row=8, col=1)

fig.update_xaxes(title_text="Pays", row=9, col=1)
fig.update_yaxes(title_text="Quantité vendue", row=9, col=1)

fig.update_xaxes(title_text="Produit", row=10, col=1)
fig.update_yaxes(title_text="Quantité vendue", row=10, col=1)

fig.update_xaxes(title_text="Client", row=11, col=1)
fig.update_yaxes(title_text="Montant total", row=11, col=1)

fig.update_xaxes(title_text="Client", row=12, col=1)
fig.update_yaxes(title_text="Nombre d'achats", row=12, col=1)

fig.update_xaxes(title_text="Client", row=13, col=1)
fig.update_yaxes(title_text="Montant total", row=13, col=1)

fig.update_xaxes(title_text="Produit", row=14, col=1)
fig.update_yaxes(title_text="Quantité vendue", row=14, col=1)

# Affichage du graphique final
fig.show()


In [ ]:
fig.write_html("rapport_analyse_donnees.html") #Creation d'un fichier HTML contenant mes graphiques

In [ ]:
FileLink("rapport_analyse_donnees.html") # affichage du lien

/content/rapport_analyse_donnees.html

In [ ]:
files.download("rapport_analyse_donnees.html")# Téléchargement du lien

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>